# Task 4
- 使用 LSTM+CRF 完成序列标注任务
- 阅读论文了解实体识别与 CRF+LSTM 做序列标注的原理

## 简单实现 LSTM 序列标注

In [1]:
# 将句子转换为 word vector
def prepare_sequence(seq, to_ix):
    idxs = []
    for w in seq:
        idxs.append(to_ix[w])
    tensor = torch.LongTensor(idxs)
    return tensor

In [2]:
# 准备训练集
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print (word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [3]:
# 创建 LSTM 序列标注类
import torch.nn as nn
import torch
import torch.nn.functional as F
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

In [4]:
# 创建模型
import torch.optim as optim
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.004)

In [5]:
for epoch in range(1,300): # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        print(loss.item())
        loss.backward()
        optimizer.step()

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


1.1119968891143799
1.1467570066452026
1.1116259098052979
1.146106481552124
1.111256718635559
1.145458459854126
1.1108894348144531
1.144813060760498
1.1105239391326904
1.1441702842712402
1.11016047000885
1.143530011177063
1.109798789024353
1.1428923606872559
1.1094390153884888
1.1422573328018188
1.1090810298919678
1.1416246891021729
1.10872483253479
1.1409947872161865
1.1083705425262451
1.1403672695159912
1.1080178022384644
1.139742374420166
1.1076672077178955
1.1391198635101318
1.1073181629180908
1.1384998559951782
1.106971025466919
1.1378822326660156
1.1066255569458008
1.1372672319412231
1.1062818765640259
1.1366546154022217
1.1059401035308838
1.1360445022583008
1.1055997610092163
1.135436773300171
1.1052614450454712
1.134831428527832
1.1049246788024902
1.1342284679412842
1.1045897006988525
1.1336278915405273
1.1042563915252686
1.1330296993255615
1.1039247512817383
1.1324340105056763
1.1035947799682617
1.131840467453003
1.1032664775848389
1.1312494277954102
1.1029399633407593
1.130660

In [6]:
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print (tag_scores)

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[-0.9594, -1.0523, -1.3177],
        [-1.1054, -0.9684, -1.2404],
        [-1.0120, -0.9800, -1.3425],
        [-0.9985, -0.9675, -1.3801],
        [-0.9888, -0.9677, -1.3941]], grad_fn=<LogSoftmaxBackward>)


In [7]:
import numpy as np
tag_scores = tag_scores.detach().squeeze(-1)
print(tag_scores)
temp = np.argmax(tag_scores, axis=1)
print(temp)

tensor([[-0.9594, -1.0523, -1.3177],
        [-1.1054, -0.9684, -1.2404],
        [-1.0120, -0.9800, -1.3425],
        [-0.9985, -0.9675, -1.3801],
        [-0.9888, -0.9677, -1.3941]])
tensor([0, 1, 1, 1, 1])


## 实现 LSTM + CRF 序列标注

这里之所以使用 CRF 来对 LSTM 的最后一层输出进行处理，是因为我们 LSTM 输出的数据具有较的随机性，而且序列标注的结果标签与标签之间的转移关系是有一定规律性的，也就需要加入新的条件满足输出对这种规律性的反应。 

CRF 的参考资料可以参考柯林斯大学的[NLP概率模型讲解](http://www.cs.columbia.edu/~mcollins/)，这部分主要是针对 NLP 领域的一些概率分布的介绍，求解 CRF 过程中使用的 Vertibi 算法可以参考[知乎点赞最高的回复](https://www.zhihu.com/question/20136144)，答主给出的回复十分通俗便于理解，同时由于这部分的代码实现的复杂性，个人独立短时间内实现太过于困难，所以 CRF 代码我参考了 github 上对 CNN+BiLSTM+CRF 论文的还原代码，通过对 CONLL2003 数据进行预处理，完成对序列标注任务;同时在这部分我也了解了[判别式模型与生成式模型的区别以及HMM与MEMM的区别](https://www.zhihu.com/question/35866596)，但是由于时间学习的时间比较短，这些东西也只是看了个大概，具体深究还得等随后在暑假期间继续进行深究。

（注：对于 CRF 的学习我想随后会进一步进行深究，因为这部分只是看懂了一个算法的大概，具体的细节部分还需要后续继续阅读相关资料补充图论与概率论的知识）


In [1]:
def to_scalar(var):
    return var.view(-1).data.tolist()[0]

# 实现argmax函数
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

计算单词与标注之间的关系：
$$ P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})} $$

In [3]:
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

概率无向图的联合概率分布：  

$$P(Y)=\frac{1}{Z(x)} \prod_{c} \psi_{c}\left(Y_{c}\right)=\frac{1}{Z(x)} \prod_{c} e^{\sum_{k} \lambda_{k} f_{k}(c, y | c, x)}=\frac{1}{Z(x)} e^{\sum_{c} \sum_{k} \lambda_{k} f_{k}\left(y_{i}, y_{i}, x, i\right)}$$

In [5]:
# 实现 BiLSTM + CRF类
from torch.autograd import Variable 
import torch.autograd
import torch.nn as nn
class BiLSTM_CRF(nn.Module):
    
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)  
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return ( Variable( torch.randn(2, 1, self.hidden_dim)),
                 Variable( torch.randn(2, 1, self.hidden_dim)) )
    
    # 实现前项算法
    def _forward_alg(self, feats):
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        forward_var = Variable(init_alphas)
        # 计算在标签传递过程中的历史值
        for feat in feats:
            alphas_t = [] 
            for next_tag in range(0,self.tagset_size):
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).unsqueeze(0))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    # 将序列通过 LSTM 处理之后的隐藏层特征值取出
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
    
    # 计算句子在当前标注下的得分
    def _score_sentence(self, feats, tags):
        score = Variable( torch.Tensor([0]) )
        tags = torch.cat( [torch.LongTensor([self.tag_to_ix[START_TAG]]), tags] )
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i+1], tags[i]] + feat[tags[i+1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
    
    # 实现 viterbi 算法
    def _viterbi_decode(self, feats):
        backpointers = []
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        forward_var = Variable(init_vvars)
        for feat in feats:
            bptrs_t = []
            viterbivars_t = [] 
            for next_tag in range(0,self.tagset_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].unsqueeze(0))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        # 处理 STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]
        # 找寻最佳路线
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG] 
        best_path.reverse()
        return path_score, best_path
    
    # 使用 log 级别的参数传递过程
    def neg_log_likelihood(self, sentence, tags):
        self.hidden = self.init_hidden()
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
    
    # 应用于进行模型运算的前项传递算法
    def forward(self, sentence): 
        self.hidden = self.init_hidden()
        lstm_feats = self._get_lstm_features(sentence)
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


为了提高标签的辨识准确度，对数据进行预处理时候，将所有的数字替换为 0 ，保证数字不对序列标注产生过大的影响；

In [6]:
import re
import codecs
# 对训练集数据进行预处理
def zero_digits(s):
    return re.sub('\d', '0', s)

# 保证读入的每一行都至少有一个单词跟一个标签
def load_sentences(path, zeros):
    sentences = []
    sentence = []
    for line in codecs.open(path, 'r', 'utf8'):
        line = zero_digits(line.rstrip()) if zeros else line.rstrip()
        if not line:
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            word = line.split()
            assert len(word) >= 2
            sentence.append(word)
    if len(sentence) > 0:
        if 'DOCSTART' not in sentence[0][0]:
            sentences.append(sentence)
    return sentences

In [7]:
# 对数据进行处理
train_sentences = load_sentences('C:/Users/WYX/Desktop/NLP训练/Task4/conll-corpora/eng.train', True)
test_sentences = load_sentences('C:/Users/WYX/Desktop/NLP训练/Task4/conll-corpora/eng.testa', True)
dev_sentences = load_sentences('C:/Users/WYX/Desktop/NLP训练/Task4/conll-corpora/eng.testb', True)

In [8]:
print(train_sentences[0])

[['EU', 'NNP', 'I-NP', 'I-ORG'], ['rejects', 'VBZ', 'I-VP', 'O'], ['German', 'JJ', 'I-NP', 'I-MISC'], ['call', 'NN', 'I-NP', 'O'], ['to', 'TO', 'I-VP', 'O'], ['boycott', 'VB', 'I-VP', 'O'], ['British', 'JJ', 'I-NP', 'I-MISC'], ['lamb', 'NN', 'I-NP', 'O'], ['.', '.', 'O', 'O']]


In [9]:
# 对标签与句子进行映射处理
START_TAG = '<START>'
STOP_TAG = '<STOP>'
def create_dico(item_list):
    assert type(item_list) is list
    dico = {}
    for items in item_list:
        for item in items:
            if item not in dico:
                dico[item] = 1
            else:
                dico[item] += 1
    return dico

def create_mapping(dico):
    sorted_items = sorted(dico.items(), key=lambda x: (-x[1], x[0]))
    id_to_item = {i: v[0] for i, v in enumerate(sorted_items)}
    item_to_id = {v: k for k, v in id_to_item.items()}
    return item_to_id, id_to_item

def word_mapping(sentences, lower):
    words = [[x[0].lower() if lower else x[0] for x in s] for s in sentences]
    dico = create_dico(words)
    dico['<UNK>'] = 10000000 
    word_to_id, id_to_word = create_mapping(dico)
    print("Found %i unique words (%i in total)" % (
        len(dico), sum(len(x) for x in words)
    ))
    print(dico['the'])
    
    return dico, word_to_id, id_to_word

# 拿出第二列用于做实体识别
def tag_mapping(sentences):
    tags = [[word[-3] for word in s] for s in sentences]
    dico = create_dico(tags)
    dico[START_TAG] = -1
    dico[STOP_TAG] = -2
    tag_to_id, id_to_tag = create_mapping(dico)
    print("Found %i unique named entity tags" % len(dico))
    print(dico)
    return dico, tag_to_id, id_to_tag

In [10]:
dico_words,word_to_id,id_to_word = word_mapping(train_sentences, True)
dico_tags, tag_to_id, id_to_tag = tag_mapping(train_sentences)

Found 17493 unique words (203621 in total)
8390
Found 47 unique named entity tags
{'NNP': 34392, 'VBZ': 2426, 'JJ': 11831, 'NN': 23899, 'TO': 3469, 'VB': 4252, '.': 7389, 'CD': 19704, 'DT': 13453, 'VBD': 8293, 'IN': 19064, 'PRP': 3163, 'NNS': 9903, 'VBP': 1436, 'MD': 1199, 'VBN': 4105, 'POS': 1553, 'JJR': 382, '"': 2178, 'RB': 3975, ',': 7291, 'FW': 166, 'CC': 3653, 'WDT': 506, '(': 2866, ')': 2866, ':': 2386, 'PRP$': 1520, 'RBR': 163, 'VBG': 2585, 'EX': 136, 'WP': 528, 'WRB': 384, '$': 427, 'RP': 528, 'NNPS': 684, 'SYM': 439, 'RBS': 35, 'UH': 30, 'PDT': 33, "''": 35, 'LS': 13, 'JJS': 254, 'WP$': 23, 'NN|SYM': 4, '<START>': -1, '<STOP>': -2}


In [11]:
# 将上步完成映射的单词拼成可进行处理的句子
# 保证所有单词均为小写状态
def lower_case(x,lower=False):
    if lower:
        return x.lower()  
    else:
        return x
def prepare_dataset(sentences, word_to_id, tag_to_id, lower=True):
    """
   处理后保证每一个项含有：
        - word indexes
        - tag indexes
    """
    data = []
    for s in sentences:
        str_words = [w[0] for w in s]
        words = [word_to_id[lower_case(w,lower) if lower_case(w,lower) in word_to_id else '<UNK>']
                 for w in str_words]
        tags = [tag_to_id[w[-3]] for w in s]
        data.append({
            'str_words': str_words,
            'words': words,
            'tags': tags,
        })
    return data

train_data = prepare_dataset(
    train_sentences, word_to_id, tag_to_id,True
)
dev_data = prepare_dataset(
    dev_sentences, word_to_id, tag_to_id, True
)
test_data = prepare_dataset(
    test_sentences, word_to_id, tag_to_id, True
)

In [12]:
print(train_data[0])

{'str_words': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'words': [944, 15473, 198, 590, 8, 3848, 207, 6233, 2], 'tags': [0, 19, 5, 1, 14, 10, 5, 1, 8]}


In [14]:
# 构建模型
import torch.optim as optim
EMBEDDING_DIM = 200 
HIDDEN_DIM = 50
model = BiLSTM_CRF(len(word_to_id), tag_to_id, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [15]:
# 测试模型是否能够调通
precheck_sent = train_data[0]['words']
print(precheck_sent)
print (model(torch.tensor(precheck_sent)))

[944, 15473, 198, 590, 8, 3848, 207, 6233, 2]
(tensor(23.7454, grad_fn=<SelectBackward>), [5, 29, 20, 37, 41, 42, 23, 28, 0])


In [51]:
# 训练模型
for epoch in range(0,1): 
    for train in train_data[0:100]:
        model.zero_grad()
        neg_log_likelihood = model.neg_log_likelihood(torch.tensor(train['words']),torch.tensor(train['tags']))
        #print(neg_log_likelihood.item())
        neg_log_likelihood.backward()
        optimizer.step()

In [52]:
_ , tag = model(torch.tensor(precheck_sent))
print(tag)
print(train_data[0]['tags'])

[0, 0, 0, 0, 14, 0, 0, 0, 8]
[0, 19, 5, 1, 14, 10, 5, 1, 8]


In [75]:
# 召回率、准确率、F1的计算
total_now = []
total_preds = []
for i in train_data[0:100]:
    total_now.extend(i['tags'])
    #print(i['tags'])
    _ , tag = model(torch.tensor(i['words']))
    #print(tag)
    total_preds.extend(tag)

In [82]:
TP = 0 
FP = 0 
FN = 0
#print((total_now))
#print((total_preds))
#print(len(total_now&total_preds))
for i in range(len(dico_tags)):
    TP = 0 
    FP = 0
    FN = 0
    for j in range(len(total_now)):
        if i == total_now[j] and i == total_preds[j] :
            #print(i)
            TP += 1
        elif i == total_now[j] and i != total_preds[j]:
            #print(i)
            FN += 1
        elif i == total_preds[j] and i != total_now[j]:
            FP += 1
    if TP + FP == 0 or TP + FN == 0 :
        continue 
    P = TP/(TP+FP)
    R = TP/(TP+FN)
    if P + R == 0 :
        continue
    F1 = 2 * P * R / (P+R)
    print('--------------')
    print('The order of tags which is in dico',i)
    print(P)
    print(R)
    print(F1)

--------------
The order of tags which is in dico 0
0.25909878682842286
0.964516129032258
0.40846994535519127
--------------
The order of tags which is in dico 1
0.4370860927152318
0.3055555555555556
0.3596730245231608
--------------
The order of tags which is in dico 2
0.5
0.023529411764705882
0.0449438202247191
--------------
The order of tags which is in dico 3
0.7083333333333334
0.4358974358974359
0.5396825396825398
--------------
The order of tags which is in dico 4
0.8080808080808081
0.7339449541284404
0.7692307692307693
--------------
The order of tags which is in dico 5
0.23529411764705882
0.037037037037037035
0.064
--------------
The order of tags which is in dico 6
0.2857142857142857
0.04040404040404041
0.07079646017699116
--------------
The order of tags which is in dico 7
1.0
0.025
0.04878048780487806
--------------
The order of tags which is in dico 8
1.0
0.967741935483871
0.9836065573770492
--------------
The order of tags which is in dico 14
0.9565217391304348
1.0
0.9777